In [10]:
# Quantum Identity Verification System (QIBS) for Carbon Data
# Based on the protocol from the paper arXiv:2403.18247v1
# Implements a Quantum Identity-Based Signature (QIBS) scheme

import math
import random
import numpy as np
from qiskit import QuantumCircuit
from qiskit.quantum_info import Statevector

# qBraid / Local Simulator setup
try:
    from qbraid import get_device
    QBRAID_AVAILABLE = True
except ImportError:
    print("⚠️ qBraid not available - using local simulator")
    from qiskit_aer import AerSimulator
    QBRAID_AVAILABLE = False
# ==============================================================================

sensor_value = 32191  # Sensor value to be converted to binary

# ==============================================================================




# ==============================================================================
#  CLASS FOR THE SECRET KEY GENERATOR (SKG) - Trusted Entity
# ==============================================================================
class SKG:
    """A trusted Secret Key Generator (SKG)."""
    def __init__(self):
        self.identities = {} # Stores keys and information for each user ID
        print("🏛️  SKG (Secret Key Generator) is online.")

    def register_user(self, user_id: str, qkd_func):
        """Generates and stores keys for a new user via QKD."""
        key_t, _ = qkd_func()
        secret_phi = random.uniform(0, 2 * math.pi)
        
        self.identities[user_id] = {
            'key': key_t,
            'phi': secret_phi
        }
        print(f"✅ User '{user_id}' registered with SKG. Key length: {len(key_t)} bits.")
        return key_t, secret_phi

# ==============================================================================
#  FUNDAMENTAL QUANTUM COMPONENTS (BB84 & QOTP)
# ==============================================================================

def generate_qkd_key_single() -> tuple:
    """
    BB84 protocol for quantum key distribution.
    Simulates sending qubits from Alice to Bob.
    """
    print("🔬 Starting BB84 protocol to generate a key...")
    bit_num = 200
    
    # Alice prepares random bits and bases
    alice_bits = np.random.randint(2, size=bit_num)
    alice_bases = np.random.randint(2, size=bit_num) # 0: Z, 1: X
    
    # Bob chooses random bases to measure
    bob_bases = np.random.randint(2, size=bit_num)
    
    qc = QuantumCircuit(bit_num, bit_num)
    
    # Alice prepares the quantum states
    for n in range(bit_num):
        if alice_bits[n] == 1:
            qc.x(n) # |1⟩
        if alice_bases[n] == 1:
            qc.h(n) # Hadamard basis
    
    qc.barrier()
    
    # Bob measures in his bases
    for m in range(bit_num):
        if bob_bases[m] == 1:
            qc.h(m)
        qc.measure(m, m)
    
    # Circuit execution
    if QBRAID_AVAILABLE:
        try:
            device = get_device("local_qiskit_simulator")
            job = device.run(qc, shots=1)
            counts = job.result().measurement_counts
        except Exception as e:
            print(f"⚠️ qBraid error: {e} - using local fallback")
            simulator = AerSimulator()
            counts = simulator.run(qc, shots=1).result().get_counts(qc)
    else:
        simulator = AerSimulator()
        counts = simulator.run(qc, shots=1).result().get_counts(qc)
    
    key = list(counts.keys())[0]
    bob_measurements = [int(bit) for bit in key[::-1]]
    
    # Public comparison of bases and key sifting
    shared_key_bits = []
    for n in range(bit_num):
        if alice_bases[n] == bob_bases[n]:
            shared_key_bits.append(alice_bits[n])
            
    qber = 1 - (np.sum(np.equal(alice_bits, bob_measurements)[alice_bases == bob_bases]) / len(shared_key_bits) if len(shared_key_bits) > 0 else 0)

    if qber < 0.11:
        print(f"✅ Secure channel (QBER={qber:.2f}). QKD key generated: {len(shared_key_bits)} bits.")
        return shared_key_bits, None
    else:
        print(f"⚠️ Possible eavesdropping detected (QBER={qber:.2f}). Aborting.")
        return [], None

def apply_qotp(qc: QuantumCircuit, message_qubits: range, key: list):
    """
    Applies the Quantum One-Time Pad to a circuit (Encryption).
    """
    num_qubits = len(message_qubits)
    if len(key) < 2 * num_qubits:
        raise ValueError("QOTP key is too short for the message.")

    for i in range(num_qubits):
        qubit_index = message_qubits[i]
        if key[2 * i] == 1: qc.z(qubit_index)
        if key[2 * i + 1] == 1: qc.x(qubit_index)
    return qc

def apply_qotp_inverse(qc: QuantumCircuit, message_qubits: range, key: list):
    """
    Applies the inverse QOTP operation (Decryption).
    """
    num_qubits = len(message_qubits)
    if len(key) < 2 * num_qubits:
        raise ValueError("QOTP key is too short for the message.")

    for i in range(num_qubits):
        qubit_index = message_qubits[i]
        if key[2 * i + 1] == 1: qc.x(qubit_index)
        if key[2 * i] == 1: qc.z(qubit_index)
    return qc

# ==============================================================================
#  QIBS PROTOCOL LOGIC (Signing and Verification)
# ==============================================================================

def signing_phase_alice(message_str: str, user_id: str, T_i: list, phi: float):
    """
    Implements the Signing Phase of the QIBS protocol.
    """
    print("✍️  ALICE: Starting Signing Phase...")
    num_qubits = len(message_str)
    
    # |P⟩: Prepare the quantum state of the message
    qc_P = QuantumCircuit(num_qubits, name="P")
    for i, bit in enumerate(message_str):
        if bit == '1':
            qc_P.x(i)
    print(f"    1. Message prepared |P⟩ = |{message_str}⟩")

    # |S⟩: Create the signature circuit from |P⟩
    qc_S = qc_P.copy(name="S")
    
    # 2. Apply the U gate transformation
    for i in range(num_qubits):
        qc_S.u(math.pi/2, phi, 0, i)
    print(f"    2. U(π/2, φ, 0) gate applied.")
    
    # 3. Apply QOTP with key T_i to generate the signature |S⟩
    apply_qotp(qc_S, range(num_qubits), T_i)
    print(f"    3. Encrypted with QOTP using key T_i to create the signature |S⟩.")
    
    return {"qc_P": qc_P, "qc_S": qc_S, "ID": user_id, "message_str": message_str}

def verification_phase_bob_skg(signature_package: dict, skg_instance: SKG, T_u: list):
    """
    Implements the Verification Phase, simulating the interaction between Bob and the SKG.
    """
    print("\n🔎 BOB & SKG: Starting Verification Phase...")
    qc_P = signature_package["qc_P"]
    qc_S = signature_package["qc_S"]
    signer_id = signature_package["ID"]
    
    print(f"    BOB: Signature package received from '{signer_id}'.")
    print(f"    BOB -> SKG: Sending |S⟩ and ID for verification...")

    print(f"    SKG: Verification request received.")
    if signer_id not in skg_instance.identities:
        print("    SKG: ❌ ERROR - Signer ID not recognized.")
        return False
    
    signer_info = skg_instance.identities[signer_id]
    T_i_retrieved = signer_info['key']
    phi_retrieved = signer_info['phi']
    
    # SKG recovers |P'⟩ by reversing the signing process on |S⟩
    qc_retrieved = qc_S.copy(name="retrieved")
    apply_qotp_inverse(qc_retrieved, range(qc_S.num_qubits), T_i_retrieved)
    for i in range(qc_S.num_qubits):
        qc_retrieved.u(math.pi/2, phi_retrieved, 0, i).inverse()
    
    print("    SKG: Decrypted |S⟩ and inverted the U gate to recover |P'⟩.")
    
    # Verification check
    original_sv = Statevector(qc_P)
    retrieved_sv = Statevector(qc_retrieved)
    is_valid = original_sv.equiv(retrieved_sv)
    
    if is_valid:
        print("    ✅ SUCCESS: The recovered state |P'⟩ matches the original state |P⟩.")
    else:
        print("    ❌ FAILURE: The states do not match. The signature is INVALID.")
    return is_valid

# ==============================================================================
#  MAIN SCRIPT EXECUTION
# ==============================================================================

if __name__ == "__main__":
    print("=" * 80)
    print("🌍 QUANTUM IDENTITY VERIFICATION PLATFORM (QIBS)")
    print("⚛️  Based on 'An Experimentally Validated Feasible Quantum Protocol...'")
    print("=" * 80)

    # === PHASE 1: INITIALIZATION ===
    print(" PHASE 1: INITIALIZATION ".center(80, "="))
    print("⚠️ WARNING: Using the same key for T_i and T_u for simulation purposes.")
    
    skg = SKG()
    alice_id = "TechGreen_Solutions_SA"
    bob_id = "Environmental_Regulator"

    # 1. Generate a single shared key using your BB84 function
    shared_key, _ = generate_qkd_key_single()
    print(f"🔑 A single shared key of {len(shared_key)} bits was generated.")

    # 2. Assign the SAME key to both users
    T_i = shared_key
    T_u = shared_key
    
    # 3. Manually register users with the SKG
    phi_alice = random.uniform(0, 2 * math.pi)
    skg.identities[alice_id] = {'key': T_i, 'phi': phi_alice}
    skg.identities[bob_id] = {'key': T_u, 'phi': 0}
    
    print(f"✅ User '{alice_id}' registered with the shared key.")
    print(f"✅ User '{bob_id}' registered with the same shared key.")
    print("-" * 80)
    
    # Message to sign (must be a binary string)
# ================================================================================================================================

    report_msg = bin(sensor_value)[2:] # Here goes the binary value
    
# ================================================================================================================================
    
    if len(T_i) < 2 * len(report_msg):
        print("❌ Error: The QKD key is too short for the message. Aborting.")
        exit()

    # === PHASE 2: SIGNING (ALICE) ===
    print("\n" + " PHASE 2: SIGNING ".center(80, "="))
    signature_package = signing_phase_alice(report_msg, alice_id, T_i, phi_alice)
    print(f"    Alice created the signature for the message |{signature_package['message_str']}⟩.")
    print("-" * 80)
    
    # === TRANSMISSION ===
    print("\n" + " 📡 TRANSMITTING SIGNATURE PACKAGE TO BOB 📡 ".center(80, "="))
    
    # === PHASE 3: VERIFICATION (BOB & SKG) ===
    print("\n" + " PHASE 3: VERIFICATION ".center(80, "="))
    is_signature_valid = verification_phase_bob_skg(signature_package, skg, T_u)
    print("-" * 80)

    print("\n" + " FINAL RESULT ".center(80, "="))
    if is_signature_valid:
        print(f"✅ The signature from '{alice_id}' for the message |{report_msg}⟩ is VALID.")
    else:
        print(f"❌ The signature is FORGED or INVALID.")
    print("=" * 80)

    # === FRAUD TEST: Tamper with the signature |S⟩ ===
    print("\n" + " 🚨 FRAUD TEST: An attacker modifies |S⟩ 🚨 ".center(80, "="))
    signature_package["qc_S"].x(0) 
    print("    An attacker applied an X gate to the first qubit of |S⟩.")
    is_tampered_valid = verification_phase_bob_skg(signature_package, skg, T_u)
    if not is_tampered_valid:
        print("\n✅ SUCCESS: The tampered signature was correctly identified as INVALID.")
    else:
        print("\n❌ FAILURE: The system accepted a forged signature.")
    print("=" * 80)


⚠️ qBraid not available - using local simulator
🌍 QUANTUM IDENTITY VERIFICATION PLATFORM (QIBS)
⚛️  Based on 'An Experimentally Validated Feasible Quantum Protocol...'
=========================== PHASE 1: INITIALIZATION ============================
⚠️ WARNING: Using the same key for T_i and T_u for simulation purposes.
🏛️  SKG (Secret Key Generator) is online.
🔬 Starting BB84 protocol to generate a key...
✅ Secure channel (QBER=0.00). QKD key generated: 109 bits.
🔑 A single shared key of 109 bits was generated.
✅ User 'TechGreen_Solutions_SA' registered with the shared key.
✅ User 'Environmental_Regulator' registered with the same shared key.
--------------------------------------------------------------------------------

=============================== PHASE 2: SIGNING ===============================
✍️  ALICE: Starting Signing Phase...
    1. Message prepared |P⟩ = |111110110111111⟩
    2. U(π/2, φ, 0) gate applied.
    3. Encrypted with QOTP using key T_i to create the signature |S